In [1]:
\l p.q

In [2]:
seq:.p.import[`keras;`:models;`:Sequential;*]
keras:.p.import `keras
dense:.p.import[`keras;`:layers;`:Dense;*]
K:.p.import[`keras;`:backend]
pd:.p.import `pandas
np:.p.import `numpy
hvd:.p.import `horovod.tensorflow

/opt/A3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/opt/A3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
skopt:.p.import `skopt
ci:.p.import[`lifelines;`:utils;*];

In [4]:
hvd[`:init;*][]

{[f;x]embedPy[f;x]}[foreign]enlist


In [5]:
nparray:.p.import[`numpy;`:array;>]

In [6]:
colStr:"IIIII"
d:(colStr;enlist ",")0: `:/home/kkumar/q/l64/hdata.csv

In [7]:
cd:cols d

In [8]:
e:"f"$d`Event;t:"f"$d`Time

In [9]:
x:"f"$flip d cd where not cd in `Event`Time

In [10]:
hvd[`:init;*]
tf:.p.import `tensorflow

k){'[y;x]}/,[enlist]|:@[{[f;x]embedPy[f;x]}].[code[code;]enlist[;;][foreign]]..


In [11]:
ev:tf[`:placeholder;*;pykwargs (`dtype;`name)!(`float64;`e)]
tm:tf[`:placeholder;*;pykwargs (`dtype;`name)!(`float64;`t)]
xv:tf[`:placeholder;*;pykwargs (`dtype;`name)!(`float64;`x)]

In [61]:
trainnet:{[data;ndl;ndn]
    nn:tf[`:layers;`:dense;*;data;3;`activation pykw tf[`:nn;`:relu]];
    / show keras[`:backend;`:get_value;<;nn];
    i:0;
    while[i<ndl;
        nn:tf[`:layers;`:dense;*;nn;ndn;`activation pykw tf[`:nn;`:relu]];
        i:i+1];
    nn:tf[`:layers;`:dense;*;nn;1;`activation pykw tf[`:nn;`:relu]];
    show keras[`:backend;`:get_value;<;nn];
    :nn}

In [66]:
fitness:{[l]
    show l[1];
    show l[2];
    nn:trainnet[tf[`:convert_to_tensor;*;nparray x til 1000];l[1];l[2]];
    nnval:keras[`:backend;`:get_value;<;nn];
    cindex:0;
    / cindex:ci[`:concordance_index;<;t;exp nnval;e];
    cost:tf[`:divide;*;tf[`:subtract;*;tf[`:transpose;*;nn];tf[`:log;*;tf[`:cumsum;*;tf[`:exp;*;nn]]]];tf[`:reduce_sum;*;e]];
    opt:tf[`:train;`:GradientDescentOptimizer;*;l[0]];
    show "here;";
    global_step:tf[`:train;`:get_or_create_global_step;*][];
    dop:hvd[`:DistributedOptimizer;*;opt];
    opt:dop[`:minimize;*;cost;`global_step pykw global_step];
    .p.e"import tensorflow as tf";
    .p.e"import horovod.tensorflow as hvd";
    .p.e"config = tf.ConfigProto()";
    .p.e"config.gpu_options.allow_growth=True";
    .p.e"config.gpu_options.visible_device_list=str(hvd.local_rank())";
    config:.p.get`config;
    print config;
    tf[`:add_to_collection;*;`prediction;nn];
    .p.e"l=list()";
    sz:count d;
    mon_sess:tf[`:Session;*;`config pykw config];
    mon_sess[`:run;*;tf[`:global_variables_initializer;*][]];

    i:0;
    .p.set[`mon_sess;mon_sess];
    .p.set[`opt;opt];
    .p.set[`cost;cost];
    .p.set[`x;xv];
    .p.set[`e;ev];
    while[i<sz;
        if[1000<=(sz-i);
            .p.set[`X;nparray x til 1000];
            .p.set[`E;nparray e til 1000];
            o:.p.eval"mon_sess.run([tf.get_collection(\"prediction\"),opt,cost],feed_dict={x:X,e:E})";
            .p.set[`o;o];
            .p.eval"l.append(o[0])"];
        i:i+1000;
        ];
        :cindex}



In [67]:
dimlr:skopt[`:space;`:Real;*;`low pykw 0.000001;`high pykw 0.01;`prior pykw `log_uniform;`name pykw `learning_rate]
dimndl:skopt[`:space;`:Integer;*;`low pykw 1;`high pykw 5;`name pykw `num_dense_layers];
dimndn:skopt[`:space;`:Integer;*;`low pykw 5;`high pykw 512;`name pykw `num_dense_nodes];


In [68]:
.p.set[`dimlr;dimlr];
.p.set[`dimndl;dimndl];
.p.set[`dimndn;dimndn]
p)dimensions=list();
p)dimensions.append(dimlr)
p)dimensions.append(dimndl)
p)dimensions.append(dimndn)


In [69]:
foo:{[l]l[0]}
.p.set[`foo;foo]
p)print(foo([10,20,30]))
/ foo[(10;20;30)]
p)print(dimensions[2])

10
Integer(low=5, high=512)


In [ ]:
/ res:skopt[`:gp_minimize;*;`func pykw fitness;`dimensions pykw dimensions;`acq_func pykw `EI;`n_calls pykw 40]
.p.set[`fitness;fitness];
p)import skopt
p)res = skopt.gp_minimize(func=fitness,dimensions=dimensions,acq_func='EI',n_calls=10)

2
140
0.580064 
0.58396  
0.580064 
0.6469944
0.5577538
0.4462031
0.5577538
0.3401164
0.5354437
0.6469944
0.5131335
0.6693046
0.6246843
0.4908234
0.3569624
0.7139249
0.5354437
0.7139249
0.6469944
0.6023741
0.5393397
0.5354437
..
"here;"
gpu_options {
  allow_growth: true
  visible_device_list: "0"
}



In [29]:
exp 0


1f
